In [141]:
import json
import torchvision
from torchvision import models
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, random_split
from torchvision import datasets
from torchvision import transforms as T
from PIL import Image
import os
import pandas as pd
import numpy as np

import rasterio
from rasterio.plot import reshape_as_image

In [117]:
class TransferLearningModel(nn.Module):
        def __init__(self, num_classes=10, pretrained=True):
            super().__init__()

            # Загружаем предобученную модель
            from torchvision import models
            backbone = models.resnet18(pretrained=pretrained)

            # Замораживаем веса (опционально)
            # for param in backbone.parameters():
            #     param.requires_grad = False

            # Заменяем последний слой
            num_features = backbone.fc.in_features
            backbone.fc = nn.Linear(num_features, num_classes)

            self.model = backbone

        def forward(self, x):
            return self.model(x)

In [109]:
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f"Обучаемые параметры: {trainable_params / 1e6:.2f}M / {total_params / 1e6:.2f}M")


Обучаемые параметры: 0.01M / 11.18M


@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

In [139]:
os.environ["KAGGLEHUB_CACHE"] = "/tmp/kagglehub_new"

path = kagglehub.dataset_download("apollo2506/eurosat-dataset")
print("Новый путь:", path)
!ls "{path}"

Using Colab cache for faster access to the 'eurosat-dataset' dataset.
Новый путь: /kaggle/input/eurosat-dataset
EuroSAT  EuroSATallBands


In [ ]:
images_root = os.path.join(path, "EuroSAT")
train_transform = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(p=.5),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

full_dataset = datasets.ImageFolder(root=images_root, transform=transform)

n = len(full_dataset)
train_size = int(0.7 * n)
val_size = int(0.15 * n)
test_size = n - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, [train_size, val_size, test_size]
)

print("✅ Данные разделены!")
print("Train:", len(train_dataset))
print("Val:", len(val_dataset))
print("Test:", len(test_dataset))
print("Классы:", full_dataset.classes)

✅ Данные разделены!
Train: 18900
Val: 4050
Test: 4050
Классы: ['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake']


In [159]:
BATCH_SIZE = 128

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

In [160]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Linear(model.fc.in_features, 10)

model = model.to(device)
print("Модель готова!")

Модель готова!


In [161]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=1e-3)

In [162]:
EPOCHS = 7
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    train_acc = 100 * correct_train / total_train
    # Валидация
    model.eval()
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()
    val_acc = 100 * correct_val / total_val

    print(f"Epoch {epoch+1}/{EPOCHS} | Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%")

Epoch 1/7 | Train Acc: 70.46% | Val Acc: 82.77%
Epoch 2/7 | Train Acc: 83.13% | Val Acc: 84.20%
Epoch 3/7 | Train Acc: 84.90% | Val Acc: 85.26%
Epoch 4/7 | Train Acc: 85.68% | Val Acc: 84.74%
Epoch 5/7 | Train Acc: 86.32% | Val Acc: 85.58%
Epoch 6/7 | Train Acc: 86.60% | Val Acc: 86.05%
Epoch 7/7 | Train Acc: 86.74% | Val Acc: 85.70%


In [163]:
model.eval()
correct_test = 0
total_test = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total_test += labels.size(0)
        correct_test += (predicted == labels).sum().item()

test_acc = 100 * correct_test / total_test
print(f"\n🎯 Тестовая точность: {test_acc:.2f}%")


🎯 Тестовая точность: 87.09%


In [164]:
torch.save(model.state_dict(), "/content/eurosat_resnet180.pth")  # Пример для Google Colab
print("✅ Модель сохранена как 'eurosat_resnet180.pth'")


✅ Модель сохранена как 'eurosat_resnet180.pth'
